<a href="https://colab.research.google.com/github/leohpark/leohpark/blob/main/Burger_and_Fries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A very simple notebook for testing out OpenAI's new function calls. Very intuitive and relatable context of a Hamburger and Fries ordering function. Allows you to see what function parameters, names, and values either assist, or detract, from your ability to make a steerable chatbot based on the "prompt" elements of a function parameter.


###Disclaimer
Unless required by applicable law or agreed to in writing, the code provided in this notebook is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

In [ ]:
!pip install openai

In [ ]:
#put your open AI key in the brackets
import openai
import json
OPENAI_KEY = ""
openai.api_key = OPENAI_KEY


In [ ]:
# confirm authentication was successful
openai.Engine.list()['data'][0]

## Function and System Message

Write your function below. Put in different object types, property names, and descriptions to see how it steers the chatbot.

My brief expereince is that Description fields greatly increase the bot's propensity to add to the Required function fields, but indiscriminately. The "Fine Tuning" seems very aggressive at matching in a "keyword" fashion without taking semantic intent into account.

Adding more description and instructions in the System Message seems to help in limiting introduction of bad/incompatible data for function passing.

Lastly, there are some blank variable assignments to clear out old values, as you will see below the code is just run in an infinite "While" loop with no end or error handling, or cleanup. There are proper ways of handling this, which I don't know.

In [ ]:
#define functions here
functions = [
        {
        "name": "hamburger_order",
        "description": "Gathers information for a custom hamburger and fries",
        "parameters": {
            "type": "object",
            "properties": {
                "double_patty": {
                    "type": "boolean",
                    "description": "whether to include two patties or one",
                },
                "burger_cooked": {
                    "type": "string",
                    "enum": ["medium rare", "medium", "medium well", "well done"],
                    "description": "The doneness of the hamburger the user asks for. Medium rare is pink in the center, medium is slightly pink. medium well is cooked through and not pink, and well done is grey and charred.",
                },
                "cheese": {"type": "boolean", "description": "Whether to include cheese.",},
                "fries": {"enum": ["no fries", "small", "medium", "large"], "description": "Only the freshest potatoes."}
                          },
            "required": ["double_patty", "burger_cooked", "cheese", "fries"]
                      },
        },
        ]

In [ ]:
#Sculpt your system messages prompts here!
#Run this to clear message variables.
messages = [ {"role": "system", "content":
              """
              You are a helpful order taking burgerbot. Greet the user and ask for their order.
              On the menu today:
                Hamburgers - Single or Double Patty. With or without cheese. Cooked to order.
                Fries - Plain potatoes only. Simple but a Classic. Comes in small, medium, or large.
              Confirm with the User any function parameters that aren't explicitly mentioned in their message.
              Confirm the order before you make a function call."""
              } ]
greetings = []
greetings_response = []
reply = []
print(f"ChatGPT: {reply}")

Here's the meat of the notebook. It greets the user, then just infinitely asks for additional info, allowing you to experiement with function data collection. When it thinks a function call is appropriate, the bot will return the Function Arguments it has collected.

You have to interrupt the while loop manually at some point in order to disengage with the bot, as there is no end/completion condition. Alternatately, the API connection may time out.

If you converse too long, you may reach the model's context limit and things may get unpredictable.

In [ ]:
# All the bot does is provide a greeting based on the System Message. If engaged on the topic of foods,
# It will generally attempt to complete an order. Once it thinks a function call is appropriate, it will
# spit out the Function Arguments, so that you can evaluate what it collected.
greetings = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    temperature= 0.3,
    messages=messages,
    functions = functions,
    max_tokens = 250,
)
greetings_response = greetings.choices[0].message.content
print(f"ChatGPT: {greetings_response}")
messages.append({"role": "assistant", "content": greetings_response})
while True:
    message = input("User : ")
    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            temperature= 0.2,
            messages=messages,
            functions = functions,
            max_tokens = 250,
        )
    reply = chat.choices[0].message.content
    reply_blob = chat["choices"][0]["message"]
    print(f"ChatGPT: {reply}")
    messages.append({"role": "assistant", "content": reply})

    if reply_blob.get("function_call"):
      function_data = chat.choices[0].message.function_call
      print(f"ChatGPT: {function_data}")